In [33]:
# Imports:
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [34]:
# Preprocessing

# Load the CSV file into a DataFrame
df = pd.read_csv('dataset.csv')

# Define the genres you are interested in
genre_filter = ['acoustic', 'electronic', 'pop', 'rock', 'folk', 'emo', 'alternative']

# Filter the DataFrame to only include rows where the Genre column matches one of the desired genres
filtered_df = df[df['track_genre'].isin(genre_filter)]

# # Save the filtered DataFrame to a new CSV file
# filtered_df.to_csv('filtered_dataset.csv', index=False)

# Calculate the distribution of the 'track_genre' column
genre_distribution = filtered_df['track_genre'].value_counts()

# Print the distribution
print(genre_distribution)

track_genre
acoustic       1000
alternative    1000
electronic     1000
emo            1000
folk           1000
pop            1000
rock           1000
Name: count, dtype: int64


In [48]:
# Load Training Dataset

# Specify the columns to drop
# Dropped all unique identifiers + duration, key, mode, and time signature due to low correlation to genre in corr matrix
columns_to_drop = ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name']

# Remove the specified columns
filteredTrainingFrame =  filtered_df.drop(columns=columns_to_drop)

# Define the mapping dictionary
genre_mapping = { 
    'acoustic': 1,
    'electronic': 2,
    'pop': 3,
    'rock': 4,
    'folk': 5,
    'emo': 6,
    'alternative': 7
    }

# Map the track_genre column to integers
filteredTrainingFrame['track_genre_int'] = filteredTrainingFrame['track_genre'].map(genre_mapping)

filteredTrainingFrame =  filteredTrainingFrame.drop(columns=['track_genre'])
print(filteredTrainingFrame.head())


   popularity  duration_ms  explicit  danceability  energy  key  loudness  \
0          73       230666     False         0.676  0.4610    1    -6.746   
1          55       149610     False         0.420  0.1660    1   -17.235   
2          57       210826     False         0.438  0.3590    0    -9.734   
3          71       201933     False         0.266  0.0596    0   -18.515   
4          82       198853     False         0.618  0.4430    2    -9.681   

   mode  speechiness  acousticness  instrumentalness  liveness  valence  \
0     0       0.1430        0.0322          0.000001    0.3580    0.715   
1     1       0.0763        0.9240          0.000006    0.1010    0.267   
2     1       0.0557        0.2100          0.000000    0.1170    0.120   
3     1       0.0363        0.9050          0.000071    0.1320    0.143   
4     1       0.0526        0.4690          0.000000    0.0829    0.167   

     tempo  time_signature  track_genre_int  
0   87.917               4              

In [49]:
# Correlation Matrix

# Compute the correlation matrix
correlation_matrix = filteredTrainingFrame.corr()

# Display the correlation matrix
print(correlation_matrix)

                  popularity  duration_ms  explicit  danceability    energy  \
popularity          1.000000     0.001977  0.098196      0.105412 -0.099522   
duration_ms         0.001977     1.000000 -0.184788     -0.156806  0.052405   
explicit            0.098196    -0.184788  1.000000      0.194339  0.047545   
danceability        0.105412    -0.156806  0.194339      1.000000  0.001293   
energy             -0.099522     0.052405  0.047545      0.001293  1.000000   
key                -0.028070    -0.005487 -0.007433      0.009245  0.048597   
loudness           -0.076680    -0.042588  0.071273      0.007374  0.771924   
mode               -0.047615    -0.005177 -0.055635     -0.115342 -0.100148   
speechiness         0.054378    -0.109326  0.303518      0.189809  0.148965   
acousticness        0.093522    -0.017288 -0.128218     -0.075042 -0.736086   
instrumentalness    0.051990     0.111727 -0.078973      0.048200 -0.017824   
liveness           -0.038797     0.064110  0.033065 

In [50]:
X = filteredTrainingFrame.drop(columns=['track_genre_int'])
y = filteredTrainingFrame['track_genre_int']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the sizes of the train and test datasets
print("Training set size:", X_train.shape, y_train.shape)
print("Testing set size:", X_test.shape, y_test.shape)


# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X.head())

Training set size: (5600, 15) (5600,)
Testing set size: (1400, 15) (1400,)
   popularity  duration_ms  explicit  danceability  energy  key  loudness  \
0          73       230666     False         0.676  0.4610    1    -6.746   
1          55       149610     False         0.420  0.1660    1   -17.235   
2          57       210826     False         0.438  0.3590    0    -9.734   
3          71       201933     False         0.266  0.0596    0   -18.515   
4          82       198853     False         0.618  0.4430    2    -9.681   

   mode  speechiness  acousticness  instrumentalness  liveness  valence  \
0     0       0.1430        0.0322          0.000001    0.3580    0.715   
1     1       0.0763        0.9240          0.000006    0.1010    0.267   
2     1       0.0557        0.2100          0.000000    0.1170    0.120   
3     1       0.0363        0.9050          0.000071    0.1320    0.143   
4     1       0.0526        0.4690          0.000000    0.0829    0.167   

     tempo 

In [53]:
# Train Logistic Regression:

# Run Logistic Regression Training and Testing
    # lr: learning rate (0 to 1.0)
    # e: epochs (iterations)
lr = 0.2
e = 100

logreg = LogisticRegression(C=lr, max_iter=e) #, class_weight='balanced')

print("training logistic regression model...")
logreg.fit(X_train_scaled, y_train)
print("done!")

# Model Statistics
print("getting statistics...")

y_trainPred = logreg.predict(X_train_scaled)

y_pred = logreg.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

print("Testing Accuracy: ")
print(accuracy)
print("done!")

training logistic regression model...
done!
getting statistics...
Testing Accuracy: 
0.4328571428571429
done!


In [54]:
# Train MLP
# Create an MLP Classifier
hiddenL = (100,)
e = 300
activationFunc = 'relu'

mlp = MLPClassifier(hidden_layer_sizes=hiddenL, max_iter=e, activation=activationFunc,
                    solver='adam', random_state=1)

# Train the MLP
mlp.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = mlp.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Testing Accuracy of the MLP model: {accuracy:.2f}")

Testing Accuracy of the MLP model: 0.60


/Users/trentgavron/Desktop/AI/project/environment/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
